In [26]:
!pip install --user -U nltk
!pip install tweet-preprocessor

Requirement already up-to-date: nltk in /home/julian/anaconda3/envs/tensorflow/lib/python3.7/site-packages (3.4.5)
  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-py3-none-any.whl size=7946 sha256=4e0bb0002232d6d704a4f39e90d7cf0d94ed999f60e373569ef37a314117839f
  Stored in directory: /home/julian/.cache/pip/wheels/af/14/89/4e427492edc4f92e8f33867aedddd879e988459b709f46c509
Successfully built tweet-preprocessor


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import re, string
import preprocessor as p

In [2]:
url = "https://archive.org/stream/ElAmorEnLosTiemposDelColera/GarciaMarquezGabriel-ElAmorEnLosTiemposDelColera_djvu.txt"
r = requests.get(url).text
soup = BeautifulSoup(r, "lxml")
text = str(soup.find_all('pre')[0])
text=str(text).replace("<pre>","")
text = text.lower()
print ('Longitud del texto: {} caracteres'.format(len(text)))

Longitud del texto: 839868 caracteres


In [3]:
def transformer(text):
  text= p.clean(text)
  text = re.sub(r'\W+',' ',text) 
  return text

word_tokens = list(text.split("\n"))

for i in range(0,len(word_tokens)):
    word_tokens[i] = transformer(word_tokens[i])
text = " ".join(word_tokens)

In [4]:
print(text[:250])

el amor en los tiempos del cólera  gabriel garcía márquez    para mercedes por supuesto     en adelanto van estos lugares  ya tienen su diosa coronada   leandro díaz    era inevitable el olor de las almendras amargas le recordaba siempre el destino d


In [5]:
import tensorflow as tf
import pickle
import numpy as np
import os
import time

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} Cáracteres únicos'.format(len(vocab)))
with open("models/vocabulario.pickle", 'wb') as handle:
  pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

35 Cáracteres únicos


In [7]:
textANum = {n : c for c,n in enumerate(vocab)}
NumAtext = np.array(vocab)
textComoNum = np.array([textANum[c] for c in text])
textComoNum
with open("models/textanum.pickle", 'wb') as handle:
  pickle.dump(textANum, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open("models/numatext.pickle", 'wb') as handle:
  pickle.dump(NumAtext, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [46]:
print ('{} ---- Caracteres a numeros ---- > {}'.format(repr(text[:13]), textComoNum[:13]))

'el amor en lo' ---- Caracteres a numeros ---- > [ 6 13  0  2 14 16 19  0  6 15  0 13 16]


In [47]:
Or_longitud = 100
muestraPorEpoca = len(text)/(Or_longitud+1)

char_dataset = tf.data.Dataset.from_tensor_slices(textComoNum)

for i in char_dataset.take(10):
  print(NumAtext[i.numpy()])

e
l
 
a
m
o
r
 
e
n


In [48]:
oraciones = char_dataset.batch(Or_longitud+1,drop_remainder=True)

for oracion in oraciones.take(5):
  print(repr(''.join(NumAtext[oracion.numpy()])))

'el amor en los tiempos del cólera  gabriel garcía márquez    para mercedes por supuesto     en adelan'
'to van estos lugares  ya tienen su diosa coronada   leandro díaz    era inevitable el olor de las alm'
'endras amargas le recordaba siempre el destino de los amores contrariados el doctor juvenal urbino lo'
' percibió desde que entró en la casa todavía en penumbras adonde había acudido de urgencia a ocuparse'
' de un caso que para él había dejado de ser urgente desde hacía muchos años el refugiado antillano je'


In [49]:
def splitEntradaSalida(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = oraciones.map(splitEntradaSalida)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [50]:
for input_example, target_example in  dataset.take(1):
  print(repr(''.join(NumAtext[input_example.numpy()])))
  print(repr(''.join(NumAtext[target_example.numpy()])))

'el amor en los tiempos del cólera  gabriel garcía márquez    para mercedes por supuesto     en adela'
'l amor en los tiempos del cólera  gabriel garcía márquez    para mercedes por supuesto     en adelan'


In [51]:
NumAtext

array([' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
       'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
       'y', 'z', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü'], dtype='<U1')

In [52]:
batch_size = 64

buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [53]:
vocab_size = len(vocab)

embedding_dim = 512

rnn_units = 256

In [35]:
def builder_model(vocab_size, embedding_dim, rnn_units,batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(
                                   vocab_size,
                                   embedding_dim, 
                                   batch_input_shape = [batch_size,None]),
                               tf.keras.layers.GRU(
                                   rnn_units,
                                   return_sequences=True, 
                                   stateful=True,
                                   recurrent_initializer = 'glorot_uniform'),
                               tf.keras.layers.LSTM(
                                   rnn_units,
                                   return_sequences=True, 
                                   stateful=True,
                                   recurrent_initializer = 'glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [36]:
model = builder_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size = batch_size
)

In [37]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predict = model(input_example_batch)
  print(example_batch_predict.shape)

(64, 100, 35)


In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 1024)          35840     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 256)           984576    
_________________________________________________________________
dense_2 (Dense)              (64, None, 35)            8995      
Total params: 1,029,411
Trainable params: 1,029,411
Non-trainable params: 0
_________________________________________________________________


In [39]:
sampled_indeces = tf.random.categorical(example_batch_predict[0],num_samples = 1)
sampled_indeces = tf.squeeze(sampled_indeces, axis = -1).numpy()
sampled_indeces

array([27,  7,  2, 10,  8,  4, 17,  7, 15, 32, 31, 11,  0, 19, 17, 20, 21,
        3, 15, 11, 28, 13, 25,  8,  5, 22, 28,  1, 15,  7, 13, 30, 16,  0,
       19,  1, 12,  8, 18, 27, 34, 27, 20,  5, 21, 24, 21, 22, 12,  5,  4,
        2, 29, 22, 27,  0, 14, 31, 27, 32,  9, 26, 30, 10, 29, 16, 14,  8,
        9,  4, 18,  3, 12, 12, 10,  9, 15, 24, 11, 23, 14, 25,  2, 22, 23,
       19, 22, 22,  6, 18, 17, 20, 21,  5,  7, 13, 26, 13, 23, 10])

In [40]:
print("Input: \n", repr("".join(NumAtext[input_example_batch[0]])))
print("Next char: \n", repr("".join(NumAtext[sampled_indeces])))


Input: 
 'rque pero cuando construyeron el templo del seminario conciliar en la manga con playa privada y ceme'
Next char: 
 'zfaigcpfnóñj rpstbnjálxgduá_nflío r_kgqzüzsdtwtukdcaéuz mñzóhyíiéomghcqbkkihnwjvmxauvruueqpstdflylvi'


In [41]:
def loss (labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits, from_logits=True)
example_batch_loss = loss(target_example_batch,example_batch_predict)
print(example_batch_predict.shape)
print(example_batch_loss.numpy().mean())

(64, 100, 35)
3.5564783


In [42]:
model.compile(optimizer='adam', loss=loss)

In [43]:
checkpoint_dir = './models'

checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt_{epoch}')

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True) 

In [44]:
epochs = 10

In [ ]:
history = model.fit(dataset,epochs = epochs, callbacks=[checkpoint_callback])

Epoch 1/10
125/125 [==============================] - 607s 5s/step - loss: 2.3997
Epoch 2/10
125/125 [==============================] - 665s 5s/step - loss: 1.8320
Epoch 3/10
125/125 [==============================] - 676s 5s/step - loss: 1.6134
Epoch 4/10
125/125 [==============================] - 763s 6s/step - loss: 1.4809
Epoch 5/10
125/125 [==============================] - 919s 7s/step - loss: 1.3951
Epoch 6/10
125/125 [==============================] - 2244s 18s/step - loss: 1.3337
Epoch 7/10
125/125 [==============================] - 17771s 142s/step - loss: 1.290113:44 - loss
Epoch 8/10
125/125 [==============================] - 48668s 389s/step - loss: 1.2555
Epoch 9/10
 16/125 [==>...........................] - ETA: 13:22:58 - loss: 1.2416

In [12]:
tf.train.latest_checkpoint(checkpoint_dir)

'./models/ckpt_8'

In [13]:
model = builder_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 512)            17920     
_________________________________________________________________
gru (GRU)                    (1, None, 256)            591360    
_________________________________________________________________
lstm (LSTM)                  (1, None, 256)            525312    
_________________________________________________________________
dense (Dense)                (1, None, 35)             8995      
Total params: 1,143,587
Trainable params: 1,143,587
Non-trainable params: 0
_________________________________________________________________


In [18]:
def generate_text(model, start_string,num_generate = 1000):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  
  # Converting our start string to numbers (vectorizing)
  input_eval = [textANum[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
 # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0
  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)
    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    # using a categorical distribution to predict the character returned by the model
    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(NumAtext[predicted_id])
  return (start_string + ''.join(text_generated))


In [23]:
print(generate_text(model, start_string="olor"))

olor un suyo había lugados del rienco y ella se le verdable y lo parecía más gracia fermina daza pero pero en el amadre tan sel extrato de vez exacitir tractos de caminos pues lo dio que una vendencia daza si espárida en la locura dispusa formo de corales muy partes a alcanzadas de años nunca y luego estaba visitarla juvenal urbino se asespresa para álicos familias y carnilachas en su mae el prísimo y le estarbas para una suyas contados a la dijuell fingién seguía enombros que habían buenas con padres la leona de las descarrarse el doctor cólores del cuples y gabriel garcía márquez el amor en además estuvo encantar la reportaba frente a encontrarse el salor y hacinderación esa cama medios y la lograso contrariarse en sábano espero sólo supuesto en ser gata ella tove el grande implutuindo sus determinas y para comprar irzubrifueron encantarle ella se conciosía dominicado así se sólo lo había madura tampoco al puerto al tenido de la falte acostaría a recibrién temprano ella casa vistos a

In [21]:
text

'el amor en los tiempos del cólera  gabriel garcía márquez    para mercedes por supuesto     en adelanto van estos lugares  ya tienen su diosa coronada   leandro díaz    era inevitable el olor de las almendras amargas le recordaba siempre el destino de los amores contrariados el doctor juvenal urbino lo percibió desde que entró en la casa todavía en penumbras adonde había acudido de urgencia a ocuparse de un caso que para él había dejado de ser urgente desde hacía muchos años el refugiado antillano jeremiah de saint amour inválido de guerra fotógrafo de niños y su adversario de ajedrez más compasivo se había puesto a salvo de los tormentos de la memoria con un sahumerio de cianuro de oro   encontró el cadáver cubierto con una manta en el catre de campaña donde había dormido siempre cerca de un taburete con la cubeta que había servido para vaporizar el veneno en el suelo amarrado de la pata del catre estaba el cuerpo tendido de un gran danés negro de pecho nevado y junto a él estaban la

In [22]:
[textANum[s] for s in "colera"]

[29, 41, 38, 31, 44, 27]

In [23]:
"fermina daza"

'fermina daza'